# DEMAND MD INTRA SPAIN

notebook to generate the file demand_ES_MD_intra.v#.csv
It makes use of the matrices generated by MobA

In [1]:
# libraries to import
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from typing import Dict
import os
print(os.getcwd())
os.chdir(r"C:\Users\LMENENDEZ\GitHub\MultiModX")
print(os.getcwd())
pd.set_option('display.max_columns', None)

c:\Users\LMENENDEZ\GitHub\MultiModX\notebooks
C:\Users\LMENENDEZ\GitHub\MultiModX


In [2]:
# O-D matrices from MobA
trips_for_demand = pd.read_csv(
    r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP4 Performance Assessment Solution\Demand data\Matrices MITMA\with_archetypes\20220922_28_trip_matrix_arc_pt_processed.csv.gz",
    compression="gzip",
    sep="|")

In [3]:
#select trips that happened on friday
trips_for_demand = trips_for_demand[trips_for_demand["date"] == 20220923].reset_index(drop=True).rename(columns={"origin_nut": "origin", "destination_nut": "destination"})

For this notebook I constructed a dictionary with the airports and their associated NUTS because I did not need information about the ports.
In the future, I could use the qgis files form MobA to associate the origine zone and the destination zone directly with the NUTS.

In [4]:
#associates each airport to the corresponding new NUTS
airports_to_NUT={"airport_LPA":("ES705","Gran Canaria"),
                 "airport_FUE":("ES704","Fuerteventura"),
                 "airport_ACE":("ES708","Lanzarote"),
                 "airport_TFS":("ES709","Tenerife"),
                 "airport_TFN":("ES709","Tenerife"),
                 "airport_GMZ": ("ES709","Tenerife"),
                 "airport_SPC":("ES707","La Palma"),
                 "airport_VDE":("ES703","El Hierro"),
                 "airport_PMI":("ES532","Mallorca"),
                 "airport_IBZ":("ES531","Eivissa i Formentera"),
                 "airport_MAH":("ES533","Menorca")}

In [5]:
def format_trips(trips: pd.DataFrame, airports_to_NUTS: dict):
    # Filter trips (only those without "abroad" as origin or destination)
    trips=trips.copy()
    trips = trips[~((trips["origin"] == "abroad") | (trips["destination"] == "abroad"))].copy()

    # Modify 'mode_tp' column: replacing modes with specific terminology
    trips.loc[:, "mode_tp"] = (
        trips["mode_sequence"]
        .str.replace("bus", "road")  # replace bus to road (some people can reach infrastructure by bus)
        .str.replace("plane", "air")  # use nomenclature of the offer data
        .str.replace("train", "rail")
    )

    # Remove "road" from the 'mode_tp' column
    trips.loc[:, "mode_tp"] = trips["mode_tp"].apply(
        lambda row: [mode for mode in row.split("-") if mode != "road"]
    )  # remove "road" (it will be considered like access time)

    # Only consider trips that do not contain "ship"
    trips = trips[~trips["mode_tp"].apply(lambda x: "ship" in x)]

    # Change aggregated island NUTS to dis-aggregated NUTS
    for key in airports_to_NUTS.keys():
        trips.loc[trips["start_node"] == key, ["origin", "origin_name"]] = [
            airports_to_NUTS[key][0],
            airports_to_NUTS[key][1]
        ]  # change start node
        trips.loc[trips["end_node"] == key, ["destination", "destination_name"]] = [
            airports_to_NUTS[key][0],
            airports_to_NUTS[key][1]
        ]  # change destination node

    return trips

In [6]:
trips_for_demand=format_trips(trips_for_demand,airports_to_NUT)

In [7]:
trips_for_demand

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp
0,20220923,P00,01002,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,NF,O,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*01002*01002*None*train_13121*00-01*road*No...,other,ES,2_48,4801303,2_48,I01_[10000-15000),A01_[25-45),male,passenger,False,4.135,139.516,1.0,0.000000,0.000000,2.067500,2.067500,0.000000,0.000000,3,road-train-road,train_13121-train_13200,train_13121,train_13200,national,2,1,0,train_13121-train_13200,train_13106,train_13200,[rail]
1,20220923,P00,01009_AM,ES211,Álava,31010_AM,ES220,Navarra,NaN,NaN,O,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01009_AM*01009_AM*None*train_11213*00-01*r...,other,ES,2_31,31010_AM,2_31,I01_[10000-15000),A01_[25-45),male,passenger,False,5.191,69.312,1.0,3.707857,0.000000,0.000000,0.494381,0.000000,0.988762,3,road-train-road,train_11213-train_11300,train_11213,train_11300,national,2,1,0,train_11213-train_11300,train_13106,train_11300,[rail]
3,20220923,P00,01036,ES211,Álava,4802006,ES213,Vizcaya,NaN,NaN,H,O,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*01036*01036*None*train_13106*00-01*road*No...,other,ES,2_01,01036,2_48,I02_[15000-inf),A02_[45-65),female,passenger,False,6.236,139.477,1.0,3.118000,0.000000,0.779500,1.559000,0.779500,0.000000,3,road-train-road,train_13106-train_13200,train_13106,train_13200,national,2,1,0,train_13106-train_13200,train_13106,train_13200,[rail]
4,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,O,NF,D04_[10000-50000),D04_[10000-50000),01-02,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_01,0105904,2_01,I02_[15000-inf),A02_[45-65),male,passenger,False,4.215,155.813,1.0,3.417568,0.113919,0.227838,0.227838,0.227838,0.000000,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200,[rail]
5,20220923,P00,0105902,ES211,Álava,09219,ES412,Burgos,NaN,NaN,W,H,D04_[10000-50000),D04_[10000-50000),00-01,train,conv_unknown,P00*0105902*0105901*None*train_11208*00-01*roa...,other,ES,2_09,09219,2_09,I01_[10000-15000),A02_[45-65),male,passenger,False,3.574,133.590,1.0,2.757086,0.000000,0.204229,0.102114,0.000000,0.510571,3,road-train-road,train_11208-train_11200,train_11208,train_11200,national,2,1,0,train_11208-train_11200,train_11208,train_11200,[rail]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227596,20220923,P23,5029709,ES243,Zaragoza,44056_AM,ES242,Teruel,NaN,NaN,NF,NF,D05_[50000-inf),D05_[50000-inf),01-02,train,conv_unknown,P23*5029709*5029703*None*train_70806*00-01*roa...,other,ES,2_22,22048,2_22,I01_[10000-15000),A02_[45-65),female,passenger,False,3.468,630.518,1.0,2.813660,0.007270,0.029082,0.152679,0.247195,0.218113,3,road-train-road,train_70806-train_67103,train_70806,train_67103,national,2,1,0,train_70806-train_67103,train_4040,train_71201,[rail]
227597,20220923,P23,5029709,ES243,Zaragoza,44056_AM,ES242,Teruel,NaN,NaN,NF,NF,D05_[50000-inf),D05_[50000-inf),01-02,train,conv_unknown,P23*5029709*5029703*None*train_70806*00-01*roa...,other,ES,2_22,22048,2_22,I01_[10000-15000),A03_[65-100),female,passenger,False,4.735,839.483,1.0,3.809162,0.026453,0.052905,0.238073,0.317430,0.290978,3,road-train-road,train_70806-train_67103,train_70806,train_67103,national,2,1,0,trai

In [8]:
#removes trips from Guadalajara to Madrid and from Madrid to Guadalajara
trips_for_demand=trips_for_demand[~(((trips_for_demand["origin"]=="ES424")&(trips_for_demand["destination"]=="ES300"))|((trips_for_demand["origin"]=="ES300")&(trips_for_demand["destination"]=="ES424")))]

In [9]:
#remove ceuta (ES630) from the destination of the trips (105 trips in total)
trips_for_demand=trips_for_demand[trips_for_demand["destination"]!="ES630"]

In [10]:
original_trips=trips_for_demand["trips"].sum()

I also need to remove the trips that contain phantom stations like I have done in the main notebook

In [11]:
# location of "ALL" train stops given by UiC
# However this list is still incomplete
stops_loc=pd.read_csv(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\infrastructure\rail_info\stops.txt").astype(str) # everything is a string here to match other formatting
stops_loc["stop_id"] = stops_loc["stop_id"].apply(lambda x: "00" + x) #to make they start with 00

In [12]:
#finds weird stations
def find_weird_stations(node_sequence,stops_loc):
    weird_stations=[]
    nodes = node_sequence.split("-")
    for node in nodes:
        if node.startswith("train_"):
            station_id=node.split("_")[1]
            if not station_id.isdigit():
                weird_stations.append(station_id)
            else:
                station_id_modified_1 = f"0071{int(station_id):05d}"
                station_id_modified_2 = f"0087{int(station_id):05d}"
                station_id_modified_3 = f"0094{int(station_id):05d}"
                if any(station_id in stops_loc["stop_id"].values for station_id in [station_id_modified_1, station_id_modified_2, station_id_modified_3]):
                    pass
                else:
                    weird_stations.append(station_id)
    return weird_stations

In [13]:
trips_for_demand = trips_for_demand.copy()
trips_for_demand.loc[:,"weird_stations"] = trips_for_demand["node_sequence_reduced"].apply(
    lambda x: find_weird_stations(x, stops_loc))

In [14]:
# Assuming the "weird_stations" column contains lists
# Flatten the lists into one combined list
all_weird_stations = trips_for_demand["weird_stations"].explode().dropna()

# Extract unique values
unique_weird_stations = all_weird_stations.unique()

# Convert back to a list if needed
unique_weird_stations = list(unique_weird_stations)
# converts this list into Moba format
unique_weird_stations_modified=[ "train_"+ station for station in unique_weird_stations] 
len(unique_weird_stations)

382

In [15]:
# read files with information about ALL stations considered by MobA
MobA_stations_coord=gpd.read_file(r"G:\Unidades compartidas\04_PROYECTOS I+D+i\2023 MultiModX\iii) Project\WP3 Scenario definition\Case study input data\Spain\v=0.7\oferta_transporte\train_stations\train_stations.shp")

In [16]:
# identifies all the un-localisable stations
nowhere_stations=set(unique_weird_stations_modified)-set(MobA_stations_coord["ID"])
print(f"there are {len(nowhere_stations)} stations that are not in the data provided by MobA but appear in the trips dataframe")

there are 197 stations that are not in the data provided by MobA but appear in the trips dataframe


In [17]:
trips_for_demand = trips_for_demand[~trips_for_demand["node_sequence_reduced"].apply(lambda x: any(station in x for station in nowhere_stations))]

In [18]:
final_trips=trips_for_demand["trips"].sum()

In [19]:
trips_small=trips_for_demand[["date","origin","destination","archetype_0","archetype_1","archetype_2","archetype_3","archetype_4","archetype_5"]]

In [20]:
print(f"In the end, we removed {original_trips-final_trips:.2f} removing trips with nowhere stations \nWe thus stay with {final_trips/original_trips*100:.2f}% of the original number of trips (after removing Guadalajara Madrid)")

In the end, we removed 8681.27 removing trips with nowhere stations 
We thus stay with 98.23% of the original number of trips (after removing Guadalajara Madrid)


In [21]:
# aggregates the archetypes
for i in range(0,6):
    globals()[f"df_{i}"]=trips_small.groupby(["date","origin","destination"], as_index = False).agg({f"archetype_{i}":"sum"})

In [22]:
dataframes = [
    (df_0,"archetype_0"),
    (df_1, 'archetype_1'),
    (df_2, 'archetype_2'),
    (df_3, 'archetype_3'),
    (df_4, 'archetype_4'),
    (df_5, 'archetype_5')
]

# Combine the DataFrames with a new "archetype" column
combined_df = pd.concat(
    [
        df.assign(archetype=archetype_col)  # Add new archetype column
        .rename(columns={archetype_col: 'trips'})  # Rename archetype column value to "value"
        for df, archetype_col in dataframes
    ],
    ignore_index=True
)

print(combined_df)


           date origin destination        trips    archetype
0      20220923  ES111       ES112   521.082997  archetype_0
1      20220923  ES111       ES113  1115.146464  archetype_0
2      20220923  ES111       ES114  4361.562918  archetype_0
3      20220923  ES111       ES130    12.683604  archetype_0
4      20220923  ES111       ES211     8.933170  archetype_0
...         ...    ...         ...          ...          ...
13057  20220923  ES709       ES618    37.994443  archetype_5
13058  20220923  ES709       ES620    13.109658  archetype_5
13059  20220923  ES709       ES704    49.866657  archetype_5
13060  20220923  ES709       ES705    92.591233  archetype_5
13061  20220923  ES709       ES708    58.934032  archetype_5

[13062 rows x 5 columns]


In [23]:
# change column order to match previous version
columns=["date","origin","destination","archetype","trips"]
combined_df=combined_df[columns]

In [24]:
# order the values to match previous version ordering
final_df=combined_df.sort_values(by=["origin","destination","archetype"])

In [25]:
# delete trips that originate abroad and finalise abroad
demand_ES_MD_intra=final_df[~((final_df["origin"]=="abroad")&(final_df["destination"]=="abroad"))]

In [26]:
# delete rows that have 0 trips
demand_ES_MD_intra=demand_ES_MD_intra[~(demand_ES_MD_intra["trips"]==0)]

In [27]:
demand_ES_MD_intra

,date,origin,destination,archetype,trips
0,20220923,ES111,ES112,archetype_0,521.082997
2177,20220923,ES111,ES112,archetype_1,9.456862
4354,20220923,ES111,ES112,archetype_2,13.669861
6531,20220923,ES111,ES112,archetype_3,52.230863
8708,20220923,ES111,ES112,archetype_4,31.313294
...,...,...,...,...,...
4353,20220923,ES709,ES708,archetype_1,24.080276
6530,20220923,ES709,ES708,archetype_2,15.388103
8707,20220923,ES709,ES708,archetype_3,142.869436
10884,20220923,ES709,ES708,archetype_4,46.082121


In [28]:
demand_ES_MD_intra.to_csv("demand_ES_MD_intra_v0.4.csv",index=False)

In [29]:
demand_ES_MD_intra[demand_ES_MD_intra["destination"]=="ES702"]

,date,origin,destination,archetype,trips


In [30]:
trips_for_demand[trips_for_demand["destination"]=="ES702"]

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,weird_stations


In [31]:
trips_for_demand[trips_for_demand["origin"]=="ES630"]

,date,trip_period,origin_zone,origin,origin_name,destination_zone,destination,destination_name,entry_point,exit_point,origin_purpose,destination_purpose,distance,route_distance,duration,mode,service,legs,trip_vehicle_type,nationality,home_census,home_zone,overnight_census,income,age,sex,vehicle_type,short_professional_driver,trips,trips_km,sample_trips,archetype_0,archetype_1,archetype_2,archetype_3,archetype_4,archetype_5,n_legs,mode_sequence,node_sequence,start_node,end_node,type,road_legs,train_legs,plane_legs,node_sequence_reduced,start_node_reduced,end_node_reduced,mode_tp,weird_stations


In [32]:
trips_for_demand[trips_for_demand["destination"]=="ES630"]["trips"].sum()

np.float64(0.0)